## Predicting literary success through writing style

Publishers and acquisition editors have a history of fancying themselves stylemakers, going with the gut, chasing a feeling when choosing what to publish. And as a result, publishers have followed the business model of operating in the red, while crossing fingers for another Dan Brown or E.L. James to pull them into the black with a runaway hit. When presented with the idea of doing market research or digging into the data, I've heard publishers dig in their heels and repeat the argument that success in the arts is unpredictable.

As the big publishing house market shrinks and new streamlined self-publishing competitors enter the field, this business model is proving increasing less viable. And in this shifting landscapehe big houses are finally beginning to turn to data. Which presents the question, can the success of a novel be predicted?

Clearly, many variables of success are out of predictive bounds -- storyline novelty, fame of the author, social trends, world events. Here, we will focus soley on the structure of the writing style and whether stylistic structure can be used to predict success.



# The Data
## The collection

For the purposes of this project, I compiled XX best-selling titles and XX titles that were less successful. 

Titles were selected from the top two selling genres, Romance and Science-Fiction, and within these genres, restricted to the sub-genres contemporary and space-opera, respectively.

The 2015 sales report from Nielsen BookScan, considered the industry standard, was used to select and categorize titles. 

* **Best-sellers** were among the top 100 selling titles for their genre in 2015. I selected titles within this group who have also won awards, The Hugo for the sci-fi titles and the RITA for romance.

* Less-successful titles (affectionately termed **Flops**) were published in the years 2013-2015 and sold fewer than 1000 copies in 2015.

## The cleaning

Using the python package textract, I wrote a function to loop through a folder of epub files, extract the text, perform some minor cleaning, a save as a text file to use as corpus documents.

In [ ]:
# function to extract text from epub
def convert_epub_to_text(epub_path, epub_file, txt_path):
    clean_text = ''
    text_name = epub_file.replace(' ','_')[:-4]+'txt' # clean up filename and change file extention
    
    text = textract.process(epub_path+epub_file,encoding='utf_8') # extract text from epub
    clean_text = text.decode('ascii', 'ignore').replace('\n',' ') # trip out the unicode and return characters

    text_file = open(txt_path+text_name, 'w') # save as text file
    text_file.write(clean_text)
    text_file.close()
    
# loop through files in directory, convert file, save file in new folder
for epub in os.listdir(epub_path):
    try:
        convert_epub_to_text(epub_path, epub, txt_path)
    except:
        print epub, " failed"

# The metrics



## The Models

In [ ]:
top n-grams

In [ ]:
additional metrics created and results from those

In [ ]:
topic modeling

In [ ]:
sentiment analysis

In [ ]:
insult lab w/MultinomialNB

In [ ]:
classification_report

In [ ]:
calibration plots

In [ ]:
Conclusion